In [2]:
import time
import requests
from selenium import webdriver
from bs4 import BeautifulSoup
from urllib.parse import urljoin
import csv
import pickle
import numpy as np
import pandas as pd
import pickle

from multiprocessing import Process
from multiprocessing import Pool
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.keys import Keys

In [3]:

options = Options()

options.add_argument('--user-data-dir=/Users/Administrator/AppData/Local/Google/Chrome/User Data/') 
options.add_argument('--profile-directory=Profile 1')
options.add_argument("--disable-extensions")
options.add_argument('--disable-blink-features=AutomationControlled')
options.add_argument("--start-maximized")

options.add_argument('disable-infobars')
options.add_experimental_option("excludeSwitches", ["enable-automation"])
options.add_experimental_option('useAutomationExtension', False)
options.add_argument("--remote-debugging-port=9222")
user_agent = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.82 Safari/537.36"
options.add_argument(f'user-agent={user_agent}')

prefs = {
    "profile.default_content_setting_values.images": 2
}
options.add_experimental_option("prefs", prefs)

# Add a preference to disable image loading


capabilities = DesiredCapabilities.CHROME.copy()
capabilities['acceptSslCerts'] = True
capabilities['acceptInsecureCerts'] = True




In [4]:
file_path = 'categories.txt'

categories = []

with open(file_path, 'r') as file:
    for line in file:     
        category = line.strip()
        categories.append(category)

In [20]:
all_details = {}
index = 0
urls = []
for count,category in enumerate(categories):
    
    driver = webdriver.Chrome(options=options , desired_capabilities=capabilities)
    driver.get("https://dir.indiamart.com/search.mp?ss=scrap&prdsrc=1&stype=attr=1|attrS&res=RC4&qu-comp=to")
    
    input_section = driver.find_element(By.XPATH,"//*[@id='search_string']")
    input_section.clear()
    time.sleep(3)
    input_section.send_keys(category)
    time.sleep(5)
    input_section.send_keys(Keys.ENTER)
    time.sleep(4)


    soup_old = ""
    while True:
        print(f'Category:{category}----Category Number:{count+1}/{len(categories)}----Dict_Length:{len(all_details)}',end='\r')
        html_text = driver.page_source
        soup = BeautifulSoup(html_text,"lxml")

        products = soup.find("div",class_='right-group flx1 df fww pr')
        sections = products.find_all("section")

        for section in sections:
            details={}
            try:
                name = section.find("a",class_="prd-name").text
                price = section.find("span",class_="elps elps2 fs14 tac clr7").text
                url = section.find("a",class_="prd-name")["href"].split(" ")[0]
                if url in urls:
                    continue

                details["name"]= name
                details["price"] = price
                details["url"] = url
                details["category"] = category
                
                urls.append(url)
                all_details[index] = details
                index+=1
            except:
                
                try:
                    name = section.find("span",class_="noPdp prd-name").text
                    price = section.find("span",class_="elps elps2 fs14 tac clr7").text
                    url = None
                    details["name"]= name
                    details["price"] = price
                    details["url"] = url
                    details["category"] = category
                    all_details[index] = details
                    index+=1
                except Exception as e:
                    
                    continue

        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        driver.implicitly_wait(10)
        
        if soup_old == soup:
            break
        else:
            
            soup_old = soup
        try:
            btn = driver.find_element(By.XPATH, "//*[@id='app']/div/main/div[4]/div[2]/div[2]/span")

            driver.execute_script("arguments[0].click();",btn)

        except:
            break
    driver.close()

In [ ]:
df = pd.DataFrame().from_dict(all_details).T
df = df.drop_duplicates()
df = df.reset_index()
df.to_csv("indiamart.csv")

In [13]:
df = pd.read_csv("indiamart.csv")
df.head()

,name,price,url,category
0,Inconel 690 Scrap,₹550/ Kilogram,https://www.indiamart.com/proddetail/inconel-6...,Industrial Scrap
1,Titanium Scrap,₹250/ Kilogram,https://www.indiamart.com/proddetail/titanium-...,Industrial Scrap
2,"Stainless Steel Plate Cut Scrap, 50 KG, Packag...",₹100/ Kg,https://www.indiamart.com/proddetail/stainless...,Industrial Scrap
3,"Black Mild Steel Scrap, For Metal Industry, Pa...",₹43/ Kg,https://www.indiamart.com/proddetail/mild-stee...,Industrial Scrap
4,"Aluminium Scrap Lithium Battery, Model Name/Nu...",₹90/ Kg,https://www.indiamart.com/proddetail/scrap-lit...,Industrial Scrap


In [14]:
df["price"] = df["price"].apply(lambda x: str(x).strip(" "))
df["price"] = df["price"].apply(lambda x: str(x).lower())

df["price"] = df["price"].apply(lambda x: str(x).replace("Kilogram","kg"))
df["price"] = df["price"].apply(lambda x: str(x).replace("Kg","kg"))
df["price"] = df["price"].apply(lambda x: str(x).replace("KG","kg"))
df["price"] = df["price"].apply(lambda x: str(x).replace("Kilogram(s)","kg"))
df["price"] = df["price"].apply(lambda x: str(x).replace("KILOGRAMS","kg"))
df["price"] = df["price"].apply(lambda x: str(x).replace("kgs","kg"))
df["price"] = df["price"].apply(lambda x: str(x).replace("kilogram","kg"))
df["price"] = df["price"].apply(lambda x: str(x).replace("kg(s)","kg"))
df["price"] = df["price"].apply(lambda x: str(x).replace("per kg","kg"))
df["price"] = df["price"].apply(lambda x: str(x).replace("kgs","kg"))

df["price"] = df["price"].apply(lambda x: str(x).replace("kg onwards","kg"))
df["price"] = df["price"].apply(lambda x: str(x).replace("Kilogram Onwards","kg"))
df["price"] = df["price"].apply(lambda x: str(x).replace("TONNE","tonne"))
df["price"] = df["price"].apply(lambda x: str(x).replace("Tonne","tonne"))
df["price"] = df["price"].apply(lambda x: str(x).replace("metric ton(s)","tonne"))
df["price"] = df["price"].apply(lambda x: str(x).replace("ton","tonne") if str(x).split("/")[-1]==" ton" else x)
df["price"] = df["price"].apply(lambda x: str(x).replace("metric ton","tonne"))

df["price"] = df["price"].apply(lambda x: str(x).replace("Ton","tonne"))



In [15]:
df["metric"] = df["price"].apply(lambda x: str(x).split("/")[-1].strip())
df["metric"].value_counts().to_dict()

{'kg': 3215,
 'piece': 365,
 'tonne': 271,
 'unit': 128,
 'pack': 33,
 'set': 27,
 'certificate': 25,
 'plant': 20,
 'project': 17,
 'hour': 16,
 'onwards': 16,
 'number': 13,
 'metric': 13,
 'service': 12,
 'packet': 11,
 'box': 8,
 'nos': 6,
 'day': 6,
 '1 set': 5,
 'dozen': 5,
 'visit': 4,
 'person': 4,
 'meter': 3,
 'piece(s)': 3,
 'month': 3,
 'annum': 3,
 'square meter': 3,
 'qty': 2,
 'pease': 2,
 "20' container": 2,
 'litre': 2,
 'course': 2,
 'certification': 2,
 '₹735': 1,
 '₹606': 1,
 '₹4.50': 1,
 '₹795': 1,
 'gram': 1,
 '₹738': 1,
 'bag': 1,
 'session': 1,
 'starting': 1,
 '1': 1,
 'carton': 1,
 'sq ft': 1,
 '50 gram': 1}

In [19]:
names_to_filter = ["kg", "tonne", "metric"]
filtered_df = df[df['metric'].isin(names_to_filter)]
filtered_df = filtered_df.reset_index(drop=True)

In [21]:
filtered_df.to_csv("filtered_indiamart.csv",index=False)

In [22]:
filtered_df

,name,price,url,category,metric
0,Inconel 690 Scrap,₹550/ kg,https://www.indiamart.com/proddetail/inconel-6...,Industrial Scrap,kg
1,Titanium Scrap,₹250/ kg,https://www.indiamart.com/proddetail/titanium-...,Industrial Scrap,kg
2,"Stainless Steel Plate Cut Scrap, 50 KG, Packag...",₹100/ kg,https://www.indiamart.com/proddetail/stainless...,Industrial Scrap,kg
3,"Black Mild Steel Scrap, For Metal Industry, Pa...",₹43/ kg,https://www.indiamart.com/proddetail/mild-stee...,Industrial Scrap,kg
4,"Aluminium Scrap Lithium Battery, Model Name/Nu...",₹90/ kg,https://www.indiamart.com/proddetail/scrap-lit...,Industrial Scrap,kg
...,...,...,...,...,...
3494,"Dull White Polyester Monofilament Yarn, For Se...",₹165/ kg,https://www.indiamart.com/proddetail/polyester...,Recycled Yarn,kg
3495,Cotton Yarn,₹110/ kg,https://www.indiamart.com/proddetail/cotton-ya...,Recycled Yarn,kg
3496,100% Egyptian Cotton Yarn,₹410/ kg,https://www.indiamart.com/proddetail/100-egypt...,Recycled Yarn,kg
3497,"Dull White Polyester Spun Yarn, For Weaving",₹100/ kg,https://www.indiamart.com/proddetail/polyester...,Recycled Yarn,kg
